# RL Exercise 5 - Evolution Strategies

**GOAL:** The goal of this exercise is to demonstrate how to use the evolution strategies (ES) algorithm.

ES is described in detail in https://arxiv.org/abs/1703.03864.

TODO: Describe ES more.

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gym
import ray
from ray.rllib.evolution_strategies import EvolutionStrategies, DEFAULT_CONFIG

Start up Ray. This must be done before we instantiate any RL algorithms. We pass in num_workers=0 because the training algorithm's constructor will create a number of actors.

In [ ]:
ray.init(num_workers=0)

Instantiate an ES algorithm object. We pass in a config object that specifies how the network and training procedure should be configured. Some of the parameters are the following.

- `num_workers` is the number of actors that the algorithm will create. This determines the degree of parallelism that will be used.
- `episodes_per_batch` is TODO
- `timesteps_per_batch` is TODO
- `noise_stdev` is TODO
- `stepsize` is TODO

In [ ]:
config = DEFAULT_CONFIG.copy()
config['num_workers'] = 3
config['episodes_per_batch'] = 100
config['timesteps_per_batch'] = 1000
config['noise_stdev'] = 0.02
config['stepsize'] = 0.01

algorithm = EvolutionStrategies('CartPole-v0', config)

**EXERCISE:** Train the algorithm for some number of steps on the CartPole environment. Compare the performance to PPO from the previous exercise.

In [ ]:
raise Exception('Implement this.')

**EXERCISE:** Instantiate an ES algorithm object on the `MountainCar-v0` environment and train it for some number of steps. Compare the performance to PPO and A3C from the previous exercise.

In [ ]:
raise Exception('Implement this.')